In [1]:
import numpy as np
import os
import sys
import h5py
import datetime
import json

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from embeddings import Embeddings


Using TensorFlow backend.


EMBEDDING(100,4,1,4) STARTED .....
Loading the embeddings from the cache
EMBEDDING(100,4,1,4) COMPLETED .....


## Setting Parameters

PLease remove the errors, fiilup all the variables.

In [4]:
model_name = 'POS_LSTM_' + '_1024_1024_' + loss_function + "_" + activation + "_" + str(window_size) + "_" + str(batch_size) #MODEL_NAME #POS-LSTM

In [3]:
word_embedding_dimension = 100
word_embedding_window_size = 4
batch_size = 128 # 32, 64, 128
epochs = 10 # 10, 15, 30
window_size = 3 # 3, 4, 5
accuracy_threshold = 0.85
activation = 'sigmoid' # sigmoid, relu, softmax
custom_accuracy = 0
loss_function = 'mse' # mse

In [5]:
embeddings = Embeddings(word_embedding_dimension, word_embedding_window_size, 1, 4)
tokenized_pos_sentences = embeddings.get_pos_categorical_indexed_sentences()
pos2index, index2pos = embeddings.get_pos_vocabulary()
no_of_unique_tags = len(pos2index)

Loading the embeddings from the cache


In [6]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_pos_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window_size]
        y = sentence[i + window_size]
        seq_in.append(x)
        seq_out.append(y)
# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  68424


In [7]:
x_data = seq_in
y_data = seq_out

In [11]:
np.array_equal(x_data[0], x_data[0])

True

In [ ]:
for i in range(len(x_data)):
    same_x = 0
    same_y = 0
    for j in range(len(x_data)):
        if np.array_equal(x_data[i], x_data[j]):
            same_x = 1 + same_x
            if np.array_equal(y_data[i], y_data[j]):
                same_y = 1 + same_y
    print(same_x, same_y)

3795 2342
3155 134
2195 170
770 447
633 20
59 19
27 5
2148 561
3795 1024
59 19
27 6
7 6
3155 695
988 350
980 305
3795 115
819 375
2195 392
1439 144
187 48
713 65
770 447
633 20
59 19
27 7
2074 924
2019 1207
1799 417
988 168
2195 585
3795 1024
1382 1126
1587 239
59 19
27 6
7 6
3155 695
988 350
980 305
661 23
114 75
310 96
93 16
770 447
633 20
59 19
8 6
65 1
59 1
3 3
751 354
2148 561
3795 2342
3155 1461
2074 286
600 454
2195 170
770 447
633 20
59 19
27 5
188 140
633 20
2195 585
3795 2342
3155 485
2195 170
770 447
2195 585
3795 2342
3155 695
988 290
2148 561
3795 1024
1382 1126
1587 319
751 175
980 404
2019 638
2148 389
1439 138
179 153
480 160
261 76
1382 1126
1587 762
1827 605
2019 638
3795 1024
1382 1126
1587 762
1827 243
600 454
2195 170
770 447
633 20
59 21
69 17
74 29
2148 561
3795 2342
188 140
633 20
59 19
146 42
713 65
770 447
770 447
2148 176
770 447
633 20
59 19
27 5
751 354
2148 561
3795 2342
3155 1461
2074 286
600 454
3795 2342
3155 1461
2074 286
600 454
2195 146
532 239
325 3

1382 156
207 186
589 161
1827 845
819 375
2195 170
770 447
633 20
59 19
27 6
17 5
566 84
577 53
260 138
2195 585
3795 1024
1382 1126
1587 319
751 175
980 404
2019 1207
1799 417
988 350
980 8
54 14
25 2
5 1
58 11
28 23
851 545
3155 695
988 350
980 404
2019 1207
1799 417
988 290
851 240
1382 1126
1587 319
751 354
2148 561
3795 1024
851 545
3155 485
770 48
532 42
65 52
77 16
988 290
2148 341
1305 574
1065 631
1799 707
2074 286
600 454
70 8
29 15
62 21
137 31
1305 202
284 200
186 17
70 56
142 27
32 28
480 160
261 161
3155 65
101 11
51 24
653 4
59 19
70 8
29 15
62 21
137 27
3795 2342
3155 166
272 118
374 60
91 55
2195 438
1468 191
260 138
351 115
137 10
532 239
325 28
55 45
480 53
91 55
28 23
819 19
59 21
69 23
94 33
3795 1024
1382 156
207 186
589 161
1468 53
105 62
286 76
851 240
577 53
260 138
851 240
1382 156
207 186
589 161
3795 2342
3155 1461
2074 655
1922 14
14 9
102 18
988 290
2148 341
1305 574
1065 325
720 553
1587 762
1827 605
2019 1207
1799 13
20 18
41 5
21 9
653 74
751 354
2148 3

2019 1207
1799 66
104 10
3795 2342
3155 1461
2074 924
2019 638
720 553
1587 26
48 34
102 18
988 290
2148 160
532 69
84 43
84 26
653 415
2195 392
1439 672
828 388
1065 631
1799 50
101 11
21 17
77 28
770 60
770 7
770 447
3795 2342
3155 695
988 350
980 404
2019 638
653 415
2195 585
3795 2342
3155 695
988 350
980 404
2019 638
720 17
31 19
34 10
119 40
2019 1207
653 415
2195 392
1439 672
828 388
1065 631
1799 50
101 11
21 17
77 28
770 115
3795 2342
3155 134
223 109
310 57
653 415
2195 585
3795 2342
3155 695
988 350
980 404
2019 638
720 17
31 19
34 10
119 40
2019 1207
1799 707
3795 1024
1382 1126
1587 762
1827 605
2019 1207
1799 417
3795 2342
3155 695
988 44
132 70
374 60
91 55
633 224
3795 2342
3155 485
770 72
339 263
480 160
261 161
1305 574
1065 631
1799 707
2074 924
2019 638
2148 561
3795 1024
1382 1126
1587 319
751 175
980 404
2019 1207
1799 66
3795 2342
3155 695
988 350
980 404
2019 1207
1799 417
653 415
2195 300
1305 574
1065 631
1799 50
101 11
21 17
77 28
770 115
3795 2342
3155 134
2

46 16
24 15
1065 325
80 46
1799 417
988 168
661 316
2148 176
770 447
633 20
59 19
661 141
980 404
2019 1207
1799 246
770 108
59 6
32 13
1382 1126
1587 319
751 354
2148 561
3795 2342
3155 695
988 350
980 404
2019 638
661 316
2148 561
3795 2342
3155 695
988 350
980 404
2019 1207
1799 707
2074 924
3795 2342
3155 695
988 350
980 8
3795 2342
3155 1461
2074 655
1922 386
710 171
980 130
600 454
653 415
2195 392
1439 138
179 153
480 108
161 68
1065 325
720 553
1587 14
32 5
29 25
38 14
1827 605
2019 1207
1799 50
101 21
58 11
28 23
819 19
59 10
26 15
62 25
1799 13
20 18
41 14
851 545
3155 695
988 290
2148 341
1305 574
1065 28
653 415
2195 170
770 23
36 29
70 1
2 1
7 3
8 2
1827 605
2019 1207
1799 707
2074 924
2019 638
2195 585
3795 2342
3155 695
988 290
2148 341
1305 82
109 20
53 38
851 545
3155 1461
2074 286
600 454
186 4
70 31
87 18
29 9
980 33
173 57
97 36
526 214
2019 638
3795 1024
1382 16
22 9
146 97
1799 246
770 108
1439 138
3795 2342
3155 1461
2074 924
2019 1207
1799 707
2074 924
2019 1
5 

26 8
39 35
3795 2342
3155 695
988 350
980 404
2019 1207
1799 101
223 109
310 3
5 2
83 4
5 3
65 9
24 5
284 200
661 316
2148 561
3795 2342
3155 485
770 13
141 19
24 17
18 7
153 63
526 214
2195 392
1439 59
78 3
36 29
70 10
91 55
633 224
1827 605
2019 638
2195 392
1439 144
187 51
526 168
1922 386
710 64
661 141
980 130
600 454
819 19
59 10
26 15
62 29
751 119
661 316
2148 176
770 447
633 224
1827 605
2019 638
720 553
1587 762
1827 243
600 454
3795 2342
3155 695
988 40
161 63
347 44
751 119
770 71
109 83
589 28
164 45
347 69
1827 605
2019 1207
1799 417
988 290
2148 389
1439 672
828 388
1065 325
2148 341
1305 202
284 44
97 78
589 161
1827 42
173 24
33 21
633 101
751 175
980 404
2019 638
3795 2342
3155 695
988 350
980 404
2019 1207
1799 417
988 350
980 130
600 454
2195 585
3795 2342
3155 1461
2074 655
1922 672
768 164
1439 672
828 388
1065 631
1799 141
272 118
374 60
91 17
109 9
72 4
18 3
74 11
284 200
768 148
1468 53
105 9
51 5
123 108
589 237
2195 300
1305 574
3795 2342
3155 1461
2074 924
2

...

3795 86
105 15
18 1
17 6
54 33
62 25
1799 246
770 108
1439 672
828 217
566 217
519 90
600 87
97 78
589 161
1827 605
2019 638
3795 1024
1382 156
207 186
4 2
1 1
114 75
310 98
88 48
2148 76
339 3
3 3
3 3
62 3
3795 2342
3155 1461
2074 924
2019 638
2148 176
770 9
12 8
3795 2342
3155 485
770 277
1305 574
1065 631
1799 101
223 109
310 60
88 32
828 388
1065 631
1799 417
55 3
21 16
44 34
56 6
164 12
50 33
589 237
2195 585
3795 1024
1382 1126
146 97
1799 417
988 27
114 75
310 98
88 4
161 5
50 9
72 61
589 237
2195 392
1439 672
828 11
2195 585
3795 2342
3155 695
988 290
3795 2342
3155 1461
2074 924
2019 638
720 553
1587 65
164 70
310 57
74 11
31 6
33 3
109 83
589 237
2195 170
770 71
109 83
3795 115
147 70
105 51
526 214
2019 638
720 553
1587 319
751 37
161 63
347 193
2195 300
1305 202
2195 170
770 140
181 31
55 40
819 222
1827 845
1922 391
519 90
600 454
3795 2342
3155 1461
2074 924
2019 1207
146 97
1799 417
988 42
117 69
286 28
188 140
633 21
113 63
310 22
13 1
7 1
53 38
819 375
2195 438
1468 36

229 22
179 153
480 160
261 161
3155 166
272 118
653 415
2195 585
3795 2342
3155 1461
2074 655
653 415
2195 585
3795 1024
1382 1126
1587 319
33 13
29 9
980 305
1922 672
768 101
1305 202
3795 1024
1382 1126
1587 319
751 37
161 63
310 57
74 11
33 13
29 9
2148 561
3795 2342
3155 1461
2074 655
1922 386
710 171
980 305
33 13
29 9
2148 561
3795 2342
3155 485
770 277
1305 202
3795 1024
1382 1126
1587 319
751 21
117 14
58 22
84 51
1799 417
988 44
132 70
374 65
161 24
851 20
147 70
105 16
713 82
653 415
2195 49
187 74
87 31
50 7
2195 300
1305 574
1065 631
1799 707
2074 48
122 15
23 11
62 3
77 36
3795 2342
3155 1461
2074 924
2019 1207
1799 707
2074 924
2019 1207
1799 246
2195 392
1439 672
828 388
770 277
1305 202
3795 2342
3155 1461
2074 924
2019 1207
1799 707
589 161
1827 845
1922 672
768 71
770 140
181 31
2195 585
3795 2342
3155 1461
2074 286
600 87
97 78
8 6
15 7
713 224
1305 202
284 200
819 222
1827 845
1922 672
768 156
3795 2342
3155 485
770 48
3795 115
147 70
105 51
526 84
1468 53
105 62
28

1439 672
1827 845
1922 672
768 148
1468 191
260 138
577 230
713 65
2195 585
3795 2342
3155 1461
3795 2342
3155 1461
2074 924
2019 1207
1799 101
223 109
310 39
56 26
633 53
2148 389
1439 165
222 63
3795 1024
1382 1126
1587 239
2195 300
1305 574
851 545
3155 1461
2074 286
600 454
819 375
2195 170
770 71
116 40
123 87
3155 1461
2074 655
1922 672
768 101
1305 574
1468 191
260 43
104 12
3795 2342
3155 695
988 350
980 305
1922 122
136 26
2148 561
3795 1024
1382 1126
1587 762
768 74
532 239
325 42
577 155
625 223
2148 38
141 90
142 27
32 28
480 108
161 11
88 17
770 447
633 224
2148 561
3795 1024
1382 1126
1587 762
661 106
661 316
2148 561
3795 2342
3155 485
770 72
577 155
625 200
980 14
43 31
70 26
65 7
78 39
577 230
713 13
141 90
142 13
65 7
78 39
77 7
179 153
480 75
1827 845
1922 386
710 365
2148 338
1468 53
1827 845
1922 672
768 164
1439 672
3795 1024
1382 1126
1587 762
116 50
146 10
260 43
3795 2342
3155 695
1827 243
600 454
819 375
2195 392
1439 672
828 217
566 217
116 50
146 10
260 138


1587 762
1827 605
2019 1207
1827 605
2019 1207
1799 707
2074 655
1922 672
768 164
1439 672
710 365
2148 561
3795 1024
1382 156
207 186
589 161
1827 605
347 44
751 119
661 316
2148 561
3795 1024
710 365
2148 561
3795 1024
1382 156
207 186
980 305
1922 672
768 164
1439 672
828 217
566 217
519 201
2019 638
2148 389
1439 672
2195 146
532 69
84 14
28 23
819 222
1827 605
2019 638
577 155
1827 845
1922 672
768 74
532 239
325 137
228 99
1065 3
2 1
1 1
577 155
625 84
661 141
980 130
600 8
29 7
108 27
519 90
2148 561
3795 2342
3155 485
1827 845
1922 155
211 97
351 115
137 18
1468 278
72 5
72 61
589 237
2195 170
3795 87
114 26
36 33
65 20
2195 300
72 61
589 237
2195 585
3795 1024
768 148
1468 38
64 2
50 4
18 3
146 42
2195 66
339 16
24 17
14 1
770 277
1305 202
3795 1024
1382 1126
1587 65
164 45
347 193
768 156
3795 2342
3155 485
770 72
339 3
851 545
2195 4
22 16
46 10
3795 2342
3155 695
988 290
2148 561
3795 2342
3155 1461
3795 2342
3155 1461
2074 924
2019 638
720 102
851 240
1382 1126
1587 319
75

661 106
661 316
2148 561
3795 1024
1382 1126
1587 239
710 365
2148 561
3795 2342
3155 1461
2074 924
2019 71
77 8
11 7
12 9
92 30
2148 341
1305 574
653 74
751 354
2148 341
1305 574
1065 631
3795 2342
3155 1461
2074 924
2019 1207
1799 707
2074 655
1922 386
710 171
980 404
2019 1207
1799 417
988 350
980 404
116 50
146 42
713 28
532 69
84 43
84 51
63 25
710 30
114 75
310 98
88 48
2148 561
3795 2342
3155 1461
2074 924
2019 1207
653 415
2195 300
1305 574
1065 631
3795 2342
3155 1461
2074 924
2019 638
720 553
1587 65
164 70
310 11
17 2
19 11
128 21
600 454
819 27
113 17
65 7
7 3
1587 239
2195 585
3795 2342
3155 166
272 118
374 98
261 161
3155 134
223 109
310 57
74 26
2148 389
1439 672
828 388
1065 325
720 553
1587 51
146 86
374 98
261 161
3155 1461
2074 924
2019 638
3795 2342
3155 1461
2074 924
2019 1207
2195 438
1468 278
526 168
1922 386
710 21
117 69
286 43
577 155
625 84
1827 845
1922 386
710 365
2148 160
532 239
325 42
577 53
260 71
63 1
44 20
286 13
77 16
187 41
625 84
661 21
1827 845
19

In [24]:
# Changes to the model to be done here
model = Sequential()
model.add(LSTM(512, input_shape=(x_data.shape[1], x_data.shape[2]), return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(512))
#model.add(Dropout(0.2))
model.add(Dense(no_of_unique_tags, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 512)            1089536   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 19)                9747      
Total params: 3,198,483
Trainable params: 3,198,483
Non-trainable params: 0
_________________________________________________________________


In [26]:
model_weights_path = "../weights/"+ model_name
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/pos_weights.{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [27]:
model_fit_summary = model.fit(x_data, y_data, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.25, callbacks=[checkpoint])

Train on 75 samples, validate on 25 samples
Epoch 1/5
Epoch 00000: saving model to ../weights/POS_LSTMmse_0_relu_3_128/pos_weights.00-0.28.hdf5
75/75 [==============================] - 0s - loss: 9.7370 - acc: 0.1867 - val_loss: 8.5557 - val_acc: 0.2800
Epoch 2/5
Epoch 00001: saving model to ../weights/POS_LSTMmse_0_relu_3_128/pos_weights.01-0.20.hdf5
75/75 [==============================] - 0s - loss: 5.1961 - acc: 0.3867 - val_loss: 5.1698 - val_acc: 0.2000
Epoch 3/5
Epoch 00002: saving model to ../weights/POS_LSTMmse_0_relu_3_128/pos_weights.02-0.16.hdf5
75/75 [==============================] - 0s - loss: 3.1124 - acc: 0.5067 - val_loss: 5.0623 - val_acc: 0.1600
Epoch 4/5
Epoch 00003: saving model to ../weights/POS_LSTMmse_0_relu_3_128/pos_weights.03-0.08.hdf5
75/75 [==============================] - 0s - loss: 3.0978 - acc: 0.3067 - val_loss: 5.0362 - val_acc: 0.0800
Epoch 5/5
Epoch 00004: saving model to ../weights/POS_LSTMmse_0_relu_3_128/pos_weights.04-0.08.hdf5
75/75 [=========

In [29]:
check_ori = 0
check_pre = 0
counter = 0
test_start = 0
test_end = 10000
list_for_hist_words = []
list_for_hist_index = []
list_for_hist_words_ori = []
list_for_hist_index_ori = []
for i in range(test_start, test_end):
    test_no = i
    to_predict = x_data[test_no:test_no+1]
    y_ans = model.predict(to_predict)
    
    for word, corr_int in pos2index.items():
        if corr_int == np.argmax(y_ans):
            #print ("pridicted: ",word, corr_int)
            check_pre = corr_int
            list_for_hist_words.append(word)
            list_for_hist_index.append(corr_int)
        if corr_int == np.argmax(y_data[test_no:test_no+1]):
            #print ("original: ",word, corr_int)
            check_ori = corr_int
            list_for_hist_words_ori.append(word)
            list_for_hist_index_ori.append(corr_int)
    if check_ori == check_pre :
        counter += 1
    #print('\n')

print("Correct predictions: ",counter, '\nTotal Predictions: ',test_end - test_start)
custom_accuracy = counter/(test_end-test_start)

Correct predictions:  19 
Total Predictions:  100


In [32]:
model_results = model_fit_summary.history
model_results.update(model_fit_summary.params)
model_results["word_embedding_dimension"] = word_embedding_dimension
model_results["word_embedding_window_size"] = word_embedding_window_size
model_results["window_size"] = window_size
model_results["batch_size"] = batch_size
model_results["epochs"] = epochs
model_results["model_name"] = model_name
model_results["accuracy_threshold"] = accuracy_threshold
model_results["activation"] = activation 
model_results["custom_accuracy"] = custom_accuracy
model_results["loss_function"] = loss_function
model_results["layers"] = []
model_results["dropouts"] = []
for layer in model.layers:
    if hasattr(layer, "units"):
        layer_summary = {}
        layer_summary["units"] = layer.get_config()["units"]
        layer_summary["name"] = layer.name
        model_results["layers"].append(layer_summary)
    if hasattr(layer, "rate"):
        dropout_summary = {}
        dropout_summary["rate"] = layer.get_config()["rate"]
        model_results["dropouts"].append(dropout_summary)
text_file_path = "../weights/{0}/model_results.json".format(model_name)
with open(text_file_path, "w") as f:
        json.dump(model_results, f)

In [15]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [16]:
# pos_vocab = [v for (k,v) in index2pos.items()]

In [17]:
# plt.figure(figsize=(16,5))
# plt.hist(list_for_hist_index, width=1, color='r', alpha=0.5)
# plt.hist(list_for_hist_index_ori, width=1, color='b', alpha=0.5)
# plt.xticks(range(len(pos_vocab)),pos_vocab, rotation='vertical')
# plt.show()

In [18]:
# list_x = []
# list_y = []
# data_all = []
# for i in range(0,1500):
#     list_x.append((index2pos[np.argmax(x_data[i][0])], index2pos[np.argmax(x_data[i][1])]))
#     list_y.append(index2pos[np.argmax(y_data[i])])
#     data_all.append((str(list_x[i]),list_y[i]))

In [19]:
# from nltk import ConditionalFreqDist as cfd
# from nltk.collocations import *
# import plotly.offline as plot
# import plotly.graph_objs as go
# plot.offline.init_notebook_mode(connected=True)
# import pandas as pd

In [20]:
# cfd_res = cfd(data_all)

In [21]:
# df = pd.DataFrame(cfd_res).fillna(0)
# mat = df.as_matrix()
# #mat

In [22]:
# trace = go.Heatmap(z = mat,
#                    x=df.columns,
#                    y=list(df.index))
# data=[trace]
# plot.iplot(data, filename='labelled-heatmap')